# XGBoost

Train and save an XGBoost model. Uses `data.csv` if present, else Iris dataset. Requires xgboost installed.


In [ ]:
# Load dataset (kaggle)
import os
import pandas as pd

try:
    from data_utils import fetch_kaggle_data
    df, chosen = fetch_kaggle_data("iabhishekofficial/mobile-price-classification")
    print(f"Loaded dataset from: {chosen} with shape {df.shape}")
except Exception as e:
    print(f"Kaggle download failed ({e}). Falling back to Iris dataset.")
    from sklearn.datasets import load_iris
    iris = load_iris(as_frame=True)
    df = iris.frame.copy()
    if 'target' not in df.columns:
        df['target'] = iris.target

# continue with existing code: set target variable name etc.

# choose target column name variable for the notebooks' subsequent cells
target = 'target'
X = df.drop(columns=[target])
y = df[target]
